# Imports


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from lenet_5 import LeNet5_5
from lenet import LeNet5
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
#from tqdm import tqdm, trange
import math
import csv
import numpy as np
import itertools
import matplotlib.pyplot as plt
import time

import scipy.stats as st


from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array, check_X_y
from sklearn.decomposition import PCA


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load Data

In [2]:
BATCH_SIZE = 256
BATCH_TEST_SIZE = 1024
data_train = MNIST('../../data/mnist',
                   download=True,
                   transform=transforms.Compose([
                       transforms.Resize((32, 32)),
                       transforms.ToTensor()]))
data_test = MNIST('../../data/mnist',
                  train=False,
                  download=True,
                  transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor()]))
data_train_loader = DataLoader(data_train, batch_size = BATCH_SIZE , shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test,  batch_size = BATCH_TEST_SIZE, num_workers=8)
data_test_loader2 = DataLoader(data_test,  batch_size = 1, num_workers=0)

TRAIN_SIZE = len(data_train_loader.dataset)
TEST_SIZE = len(data_test_loader.dataset)
NUM_BATCHES = len(data_train_loader)
NUM_TEST_BATCHES = len(data_test_loader)



# Create Model and load weights

In [3]:
model_original = LeNet5()
model_original.load_state_dict(torch.load("LeNet-saved"))
criterion = nn.NLLLoss()

# Test

In [4]:
def validate(net, criterion):
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(data_test_loader2):
    
        output = net(images)
        avg_loss += criterion(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(data_test)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_test)))
    return float(total_correct) / len(data_test)

In [5]:
def validate_noisy (net, criterion):
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(data_test_loader2):
        
        output, _ = net(images)
        avg_loss += criterion(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(data_test)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_test)))
    return float(total_correct) / len(data_test)

In [6]:
def validate_noisy_mask (net, criterion, mask):
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(data_test_loader2):
        
        output, _ = net(images, mask)
        avg_loss += criterion(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(data_test)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_test)))
    return float(total_correct) / len(data_test)

In [7]:
def train_above_5(net, criterion, optimizer,coef, increase = False):
    
    net.train()
    avg_loss = 0
    total_correct=0
    for i, (images, labels) in enumerate(data_train_loader):
        net.zero_grad()
        labels = (labels > 5).long()
        output = net(images)
        if increase:
            loss = criterion(output, labels) + coef* 1/(torch.sum(net.intermed.scales()) )
        else:
            loss = criterion(output, labels) #+ 1/(torch.std(net.intermed.noise) )
        avg_loss += loss 
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()
        loss.backward()
        optimizer.step()
        #print("here")
    avg_loss /= len(data_train)
    print('Train Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_train)))
    return float(float(total_correct) / len(data_train))



In [8]:
def train_noisy(net, criterion, optimizer,coef=1, increase = False):
    SNR_decoded = []
    cnt = 0
    
    
    net.train()
    avg_loss = 0
    total_correct=0
    for i, (images, labels) in enumerate(data_train_loader):
        net.zero_grad()
        
        output,noisy = net(images)
        
        SNR_decoded.append((((images**2).mean())/(abs(images-noisy)).var()).item())
        if increase:
            loss1 =  criterion(output, labels)
            loss2 = torch.log(torch.mean(net.intermed.scales()) )
            
            loss = loss1 - coef*loss2

        else:
            loss = criterion(output, labels) #+ 1/(torch.std(net.intermed.noise) )
        
        avg_loss += loss 
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()
        loss.backward()
        optimizer.step()
        cnt =+ 1
        del noisy
        del output
        
        
        #print("here")
    avg_loss /= len(data_train)
    print(sum(SNR_decoded)/cnt)
    print('Train Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_train)))
    return float(float(total_correct) / len(data_train))




In [9]:
def train_noisy_mask(net, criterion, optimizer, mask,coef=1, increase = False):
    SNR_decoded = []
    cnt = 0
    
    
    net.train()
    avg_loss = 0
    total_correct=0
    for i, (images, labels) in enumerate(data_train_loader):
        net.zero_grad()
        
        output,noisy = net(images, mask)
        
        SNR_decoded.append((((images**2).mean())/(abs(images-noisy)).var()).item())
        if increase:
            loss1 =  criterion(output, labels)
            loss2 = torch.log(torch.mean(net.intermed.scales()) )
            
            loss = loss1 - coef* loss2

        else:
            loss = criterion(output, labels) #+ 1/(torch.std(net.intermed.noise) )
        
        avg_loss += loss 
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()
        loss.backward()
        optimizer.step()
        cnt =+ 1
        del noisy
        del output
        
        
        #print("here")
    avg_loss /= len(data_train)
    print(sum(SNR_decoded)/cnt)
    print('Train Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_train)))
    return float(float(total_correct) / len(data_train))





In [10]:
def validate_stochastic(net, criterion, times=100):
    sum = 0
    lis = []
    for i in range (times):
        acc = validate(net, criterion)
        sum += acc
        lis.append(acc)
    print("avg is:", sum/times)
    return(np.array(lis).std)
        

In [11]:
def validate_stochastic_noisy(net, criterion, times=100):
    sum = 0
    lis = []
    for i in range (times):
        acc = validate_noisy(net, criterion)
        sum += acc
        lis.append(acc)
    print("avg is:", sum/times)
    return(np.array(lis).std)
        

In [12]:
def validate_stochastic_noisy_mask(net, criterion, mask, times=100):
    sum = 0
    lis = []
    for i in range (times):
        acc = validate_noisy_mask(net, criterion, mask)
        sum += acc
        lis.append(acc)
    print("avg is:", sum/times)
    return(np.array(lis).std)
        

In [13]:
#validate (model_original, criterion)
validate (model_original, criterion)

Test Avg. Loss: 0.034585, Accuracy: 0.990500


0.9905

# Get conv shapes and layers

In [14]:
print(model_original.convnet)

Sequential(
  (c1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (s2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (c3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu3): ReLU()
  (s4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (c5): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (relu5): ReLU()
)


In [25]:
def get_conv_indices(net):
    conv_layers = []
    fc_layers=[]
    for i, layer in enumerate(net.convnet):
        if isinstance(layer, nn.Conv2d):
            if ((i is not 0 )):
                conv_layers.append(i)
    conv_layers.append(len(net.convnet))
    for i, layer in enumerate(net.fc):
        if isinstance(layer, nn.Linear):
            fc_layers.append(i)

    #conv_layers.append(-2)
    print (conv_layers, fc_layers)
    
    return conv_layers



In [26]:
def get_conv_shapes(net):
    conv_shapes=[]
    for cnt2, (data, target) in enumerate(data_test_loader):
        for cnt,i in enumerate(conv_layers):
            #newmodel = torch.nn.Sequential(*(list(model_test.features)[0:i]))
            newmodel_original =  torch.nn.Sequential(*(list(net.convnet)[0:i]))

            output_original = newmodel_original(data)
            conv_shapes.append(output_original.shape[1:])
            print (output_original.shape[1:])
        if (cnt2==0):
            conv_shapes.append(data.shape[1:])
            print(data.shape[1:])
            break

    return conv_shapes

In [27]:
conv_layers = get_conv_indices(model_original)
conv_shapes = get_conv_shapes(model_original)

[3, 6, 8] [0, 2]
torch.Size([6, 14, 14])
torch.Size([16, 5, 5])
torch.Size([120, 1, 1])
torch.Size([1, 32, 32])


# Noisy network

In [28]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-10) #-inf

        #self.noise = nn.Parameter(torch.Tensor(size).normal_(mean=prior_mus, std=prior_sigmas))
        self.normal = torch.distributions.normal.Normal(0,1)

        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self, mask):
        epsilon = self.normal.sample(self.rhos.shape)*mask
        return self.locs + self.scales() * epsilon
                                 
                            
                            
    def forward(self, input, mask):
        noise = self.sample_noise(mask)
        return (input)*mask + noise


# In[61]:


class LeNet_syn(nn.Module):

    def __init__(self, model_features, model_classifier,index, min_scale, max_scale, given_locs, given_scales):
        super(LeNet_syn, self).__init__()
        
                                
        self.model_pt1 =  torch.nn.Sequential(*(list(model_features)[0:conv_layers[index]]))
        self.intermed = NoisyActivation(given_locs, given_scales, min_scale, max_scale)
        self.model_pt2 =  torch.nn.Sequential(*(list(model_features)[conv_layers[index]:]))
        self.model_pt3 = model_classifier

        for child in itertools.chain(self.model_pt1, self.model_pt2, self.model_pt3): #self.model_pt2 #(self.model_pt2, 
            for param in child.parameters():
                param.requires_grad = False
            if isinstance(child, nn.modules.batchnorm._BatchNorm):
                child.eval()
                child.affine = False
                child.track_running_stats = False
                
        #self.intermed.rhos.reuires_grad = True
        #self.intermed.locs.reuires_grad = True
                                 
    def forward(self, img, mask):
                                 
        x = self.model_pt1(img)
        x = self.intermed(x, mask)
        noisy = x.detach()

        x = self.model_pt2(x)                    
        x = x.view(img.size(0), -1)
        x = self.model_pt3(x)                                 

        return x, noisy
    



In [29]:

model_original = LeNet5()
model_original.load_state_dict(torch.load("LeNet-saved"))
criterion = nn.NLLLoss()
mus = torch.zeros((1,120,1,1))
scale = torch.ones((1,120,1,1))


In [30]:
model_syn = LeNet_syn(model_original.convnet, model_original.fc,2 ,0.0001, 3 ,mus, scale )
model_syn.load_state_dict(torch.load("lenet-last-layer-cloak",  map_location="cpu"))

In [31]:
#model_syn.intermed.rhos.requires_grad = False 

In [32]:
print(len(list(filter(lambda p: p.requires_grad, model_syn.parameters()))))

2


In [33]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_syn.parameters()), lr=0.0001, weight_decay=0)

In [34]:
mult_mask = torch.ones(model_syn.intermed.rhos.shape)


In [ ]:
for epoch in range(5): #45 0.01 10 -finess 0.1 coef lr 0.001/ 15 lr 0.01 coef 1 / 20 coef1 lr 0.01/
    print(torch.mean(model_syn.intermed.scales()))
    print(torch.max(model_syn.intermed.scales()))
    print(torch.min(model_syn.intermed.scales()))
    train_noisy_mask(model_syn, criterion, optimizer,mult_mask, 1,True)
    print("*******************************************************")
    #print(list(filter(lambda p: p.requires_grad, model_syn.parameters())))

In [40]:
validate_stochastic_noisy_mask(model_syn, criterion,mult_mask, 20) #40

Test Avg. Loss: 0.181763, Accuracy: 0.950400
Test Avg. Loss: 0.184854, Accuracy: 0.950000
Test Avg. Loss: 0.179758, Accuracy: 0.950400
Test Avg. Loss: 0.186991, Accuracy: 0.949800
Test Avg. Loss: 0.176966, Accuracy: 0.949200
Test Avg. Loss: 0.174475, Accuracy: 0.951900
Test Avg. Loss: 0.183070, Accuracy: 0.947800
Test Avg. Loss: 0.192468, Accuracy: 0.947800
Test Avg. Loss: 0.184152, Accuracy: 0.953500
Test Avg. Loss: 0.186166, Accuracy: 0.948300
Test Avg. Loss: 0.165158, Accuracy: 0.954200
Test Avg. Loss: 0.190570, Accuracy: 0.950800
Test Avg. Loss: 0.178145, Accuracy: 0.949800
Test Avg. Loss: 0.182152, Accuracy: 0.948300
Test Avg. Loss: 0.173242, Accuracy: 0.951600
Test Avg. Loss: 0.188251, Accuracy: 0.947900
Test Avg. Loss: 0.175125, Accuracy: 0.947900
Test Avg. Loss: 0.185535, Accuracy: 0.949500
Test Avg. Loss: 0.182943, Accuracy: 0.951200
Test Avg. Loss: 0.185052, Accuracy: 0.950700
avg is: 0.9500500000000001


<function ndarray.std>

# Save for MI

In [45]:
for i, (images, labels) in enumerate(data_test_loader2):
    x= images
    

In [46]:
imgs =np.reshape(np.squeeze(images.detach().numpy()), (1,-1))
lbls = np.reshape(np.squeeze(labels.detach().numpy()), (1,-1))
encoded_noise = model_syn.intermed.rhos.detach().numpy()
encoded_original = model_syn.intermed.rhos.detach().numpy()
image_noisy =np.reshape(np.squeeze(model_syn.intermed.rhos.detach().numpy()), (1,-1))
total_correct = 0

model_syn.eval()
for i, (images, labels) in enumerate(data_test_loader2):
    
    #labels = (labels > 5).long()
    imgs=np.concatenate((imgs,np.reshape(np.squeeze(images.detach().numpy()), (1,-1)) ))
    np.save("original-image-mutual_info-2class-input-noise-last-layer-cloak", imgs)
    lbls=np.concatenate((lbls,np.reshape(np.squeeze(labels.detach().numpy()), (1,-1)) ))
    np.save("original-labels-mutual_info-2class-input-noise-last-layer-cloak", lbls)
    
    
    x= model_syn.model_pt1(images)
    x = model_syn.intermed(x,mult_mask)
   
    image_noisy=np.concatenate((image_noisy,np.reshape(np.squeeze(x.detach().numpy()), (1,-1)) ))
    np.save("noisy-image-mutual_info-2class-input-noise-last-layer-cloak", image_noisy)

    #print(imgs.shape, lbls.shape, encoded_original.shape, encoded_noise.shape, image_noisy.shape)
    
    x = model_syn.model_pt2(x)                    
    x = x.view(images.size(0), -1)
    output = model_syn.model_pt3(x)                                 


    
    #print(imgs.shape, image_noisy.shape)

    pred = output.detach().max(1)[1]
    total_correct += pred.eq(labels.view_as(pred)).sum()


print('Test Avg. Loss: %f, Accuracy: %f' % (2, float(total_correct) / len(data_test)))
print (float(total_correct) / len(data_test))

Test Avg. Loss: 2.000000, Accuracy: 0.948600
0.9486
